In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold,GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [3]:
train = pd.read_csv("abalone_train.csv",index_col = 0)
train


,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,,
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
90610,M,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,6
90611,M,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,9
90612,I,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,6


In [4]:
testdf = pd.read_csv("abalone_test.csv",index_col = 0)

In [11]:
# as our y is multi-class col == use le;

le =LabelEncoder()
y = le.fit_transform(train["Rings"])
le.classes_

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29], dtype=int64)

In [12]:
# define X:
# as we can see "sex" col == > catergorical data ==> convert into numerical

X = train.drop(["Rings"],axis = 1)
# crete dumies:
dum_X = pd.get_dummies(X,drop_first = True)   
dum_X 

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Sex_I,Sex_M
id,,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,False,False
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,False,False
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,True,False
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,False,True
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,True,False
...,...,...,...,...,...,...,...,...,...
90610,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,False,True
90611,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,False,True
90612,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,True,False


In [5]:
################################### Stacking ######################################

In [8]:
cat = CatBoostRegressor(random_state = 24)
rf = RandomForestRegressor(random_state = 24)
xgb = XGBRegressor(random_state = 24)
lgb = LGBMRegressor(random_state = 24)

In [13]:
# define stack

stack = StackingRegressor([("RF", rf), ("XGB", xgb), ("LGB", lgb)], final_estimator = cat, passthrough = True)

stack.fit(dum_X, y)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002276 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1335
[LightGBM] [Info] Number of data points in the train set: 90615, number of used features: 9
[LightGBM] [Info] Start training from score 8.696529
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000807 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1330
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 9
[LightGBM] [Info] Start training from score 8.704947
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [In

123:	learn: 1.8189880	total: 1.64s	remaining: 11.6s
124:	learn: 1.8189190	total: 1.65s	remaining: 11.6s
125:	learn: 1.8188843	total: 1.67s	remaining: 11.6s
126:	learn: 1.8187483	total: 1.68s	remaining: 11.6s
127:	learn: 1.8186798	total: 1.69s	remaining: 11.5s
128:	learn: 1.8186218	total: 1.71s	remaining: 11.5s
129:	learn: 1.8185622	total: 1.72s	remaining: 11.5s
130:	learn: 1.8184409	total: 1.73s	remaining: 11.5s
131:	learn: 1.8182860	total: 1.74s	remaining: 11.4s
132:	learn: 1.8181426	total: 1.75s	remaining: 11.4s
133:	learn: 1.8181152	total: 1.76s	remaining: 11.4s
134:	learn: 1.8180071	total: 1.77s	remaining: 11.3s
135:	learn: 1.8177574	total: 1.78s	remaining: 11.3s
136:	learn: 1.8176752	total: 1.79s	remaining: 11.3s
137:	learn: 1.8175198	total: 1.8s	remaining: 11.3s
138:	learn: 1.8171044	total: 1.81s	remaining: 11.2s
139:	learn: 1.8168641	total: 1.82s	remaining: 11.2s
140:	learn: 1.8167691	total: 1.83s	remaining: 11.2s
141:	learn: 1.8163935	total: 1.84s	remaining: 11.2s
142:	learn: 1

289:	learn: 1.7912954	total: 3.51s	remaining: 8.6s
290:	learn: 1.7911033	total: 3.52s	remaining: 8.59s
291:	learn: 1.7908834	total: 3.54s	remaining: 8.57s
292:	learn: 1.7906705	total: 3.55s	remaining: 8.56s
293:	learn: 1.7905771	total: 3.56s	remaining: 8.54s
294:	learn: 1.7904475	total: 3.57s	remaining: 8.53s
295:	learn: 1.7903683	total: 3.58s	remaining: 8.52s
296:	learn: 1.7902969	total: 3.59s	remaining: 8.5s
297:	learn: 1.7902790	total: 3.6s	remaining: 8.48s
298:	learn: 1.7901578	total: 3.61s	remaining: 8.47s
299:	learn: 1.7900047	total: 3.63s	remaining: 8.46s
300:	learn: 1.7898185	total: 3.64s	remaining: 8.45s
301:	learn: 1.7897181	total: 3.65s	remaining: 8.44s
302:	learn: 1.7895755	total: 3.66s	remaining: 8.43s
303:	learn: 1.7894532	total: 3.68s	remaining: 8.42s
304:	learn: 1.7892855	total: 3.69s	remaining: 8.41s
305:	learn: 1.7890889	total: 3.7s	remaining: 8.4s
306:	learn: 1.7889743	total: 3.72s	remaining: 8.39s
307:	learn: 1.7888951	total: 3.73s	remaining: 8.38s
308:	learn: 1.788

462:	learn: 1.7664361	total: 5.4s	remaining: 6.26s
463:	learn: 1.7662993	total: 5.41s	remaining: 6.25s
464:	learn: 1.7662102	total: 5.42s	remaining: 6.23s
465:	learn: 1.7661213	total: 5.43s	remaining: 6.22s
466:	learn: 1.7660576	total: 5.44s	remaining: 6.21s
467:	learn: 1.7659398	total: 5.45s	remaining: 6.2s
468:	learn: 1.7658289	total: 5.46s	remaining: 6.18s
469:	learn: 1.7657760	total: 5.47s	remaining: 6.17s
470:	learn: 1.7656665	total: 5.49s	remaining: 6.16s
471:	learn: 1.7655687	total: 5.5s	remaining: 6.15s
472:	learn: 1.7653469	total: 5.51s	remaining: 6.14s
473:	learn: 1.7652125	total: 5.52s	remaining: 6.13s
474:	learn: 1.7650186	total: 5.53s	remaining: 6.12s
475:	learn: 1.7649326	total: 5.54s	remaining: 6.1s
476:	learn: 1.7647853	total: 5.56s	remaining: 6.09s
477:	learn: 1.7646553	total: 5.57s	remaining: 6.08s
478:	learn: 1.7644498	total: 5.58s	remaining: 6.07s
479:	learn: 1.7643050	total: 5.59s	remaining: 6.05s
480:	learn: 1.7641841	total: 5.6s	remaining: 6.04s
481:	learn: 1.764

623:	learn: 1.7447749	total: 7.25s	remaining: 4.37s
624:	learn: 1.7445154	total: 7.27s	remaining: 4.36s
625:	learn: 1.7443736	total: 7.29s	remaining: 4.35s
626:	learn: 1.7443309	total: 7.3s	remaining: 4.34s
627:	learn: 1.7441473	total: 7.31s	remaining: 4.33s
628:	learn: 1.7440679	total: 7.32s	remaining: 4.32s
629:	learn: 1.7439728	total: 7.33s	remaining: 4.3s
630:	learn: 1.7437935	total: 7.34s	remaining: 4.29s
631:	learn: 1.7436787	total: 7.35s	remaining: 4.28s
632:	learn: 1.7436213	total: 7.37s	remaining: 4.27s
633:	learn: 1.7433975	total: 7.38s	remaining: 4.26s
634:	learn: 1.7432749	total: 7.39s	remaining: 4.25s
635:	learn: 1.7432112	total: 7.41s	remaining: 4.24s
636:	learn: 1.7431075	total: 7.42s	remaining: 4.23s
637:	learn: 1.7429854	total: 7.43s	remaining: 4.21s
638:	learn: 1.7428121	total: 7.44s	remaining: 4.2s
639:	learn: 1.7426943	total: 7.46s	remaining: 4.19s
640:	learn: 1.7424815	total: 7.47s	remaining: 4.18s
641:	learn: 1.7423247	total: 7.48s	remaining: 4.17s
642:	learn: 1.7

790:	learn: 1.7240855	total: 9.13s	remaining: 2.41s
791:	learn: 1.7240500	total: 9.14s	remaining: 2.4s
792:	learn: 1.7239041	total: 9.15s	remaining: 2.39s
793:	learn: 1.7237837	total: 9.16s	remaining: 2.38s
794:	learn: 1.7237009	total: 9.17s	remaining: 2.36s
795:	learn: 1.7235500	total: 9.18s	remaining: 2.35s
796:	learn: 1.7233889	total: 9.19s	remaining: 2.34s
797:	learn: 1.7232149	total: 9.2s	remaining: 2.33s
798:	learn: 1.7231961	total: 9.21s	remaining: 2.32s
799:	learn: 1.7230328	total: 9.22s	remaining: 2.31s
800:	learn: 1.7229733	total: 9.23s	remaining: 2.29s
801:	learn: 1.7228662	total: 9.24s	remaining: 2.28s
802:	learn: 1.7227343	total: 9.25s	remaining: 2.27s
803:	learn: 1.7226343	total: 9.26s	remaining: 2.26s
804:	learn: 1.7223377	total: 9.27s	remaining: 2.25s
805:	learn: 1.7222067	total: 9.28s	remaining: 2.23s
806:	learn: 1.7220574	total: 9.29s	remaining: 2.22s
807:	learn: 1.7218742	total: 9.31s	remaining: 2.21s
808:	learn: 1.7216830	total: 9.32s	remaining: 2.2s
809:	learn: 1.7

959:	learn: 1.7042816	total: 11s	remaining: 460ms
960:	learn: 1.7042312	total: 11s	remaining: 448ms
961:	learn: 1.7041034	total: 11.1s	remaining: 437ms
962:	learn: 1.7039980	total: 11.1s	remaining: 426ms
963:	learn: 1.7038713	total: 11.1s	remaining: 414ms
964:	learn: 1.7037613	total: 11.1s	remaining: 403ms
965:	learn: 1.7037201	total: 11.1s	remaining: 391ms
966:	learn: 1.7036286	total: 11.1s	remaining: 380ms
967:	learn: 1.7034864	total: 11.1s	remaining: 368ms
968:	learn: 1.7033467	total: 11.1s	remaining: 357ms
969:	learn: 1.7032635	total: 11.2s	remaining: 345ms
970:	learn: 1.7031143	total: 11.2s	remaining: 334ms
971:	learn: 1.7029809	total: 11.2s	remaining: 322ms
972:	learn: 1.7028928	total: 11.2s	remaining: 311ms
973:	learn: 1.7028466	total: 11.2s	remaining: 299ms
974:	learn: 1.7027624	total: 11.2s	remaining: 288ms
975:	learn: 1.7025935	total: 11.2s	remaining: 276ms
976:	learn: 1.7024634	total: 11.2s	remaining: 265ms
977:	learn: 1.7023563	total: 11.3s	remaining: 253ms
978:	learn: 1.70

StackingRegressor(estimators=[('RF', RandomForestRegressor(random_state=24)),
                              ('XGB',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=24, ...)),
                              ('LGB', LGBMRegressor(random_state=24))],
                  final_estimator=<catboost.core.CatBoostRegressor object at 0x00000229C84D5F50>,
                  passthrough=True)

In [15]:
# create dummies of test

dum_tst = pd.get_dummies(testdf,drop_first = True)

# find y_pred
y_pred = stack.predict(dum_tst)

res = pd.DataFrame({"id": list(testdf.index), "Rings": y_pred})
res

,id,Rings
0,90615,8.853000
1,90616,8.598602
2,90617,9.140961
3,90618,9.590303
4,90619,6.659617
...,...,...
60406,151021,5.321067
60407,151022,8.404619
60408,151023,11.050161
60409,151024,12.347733


In [16]:
(res["Rings"] < 0).any().any()

False

In [17]:
res.to_csv("sbt_abalone_05.csv", index = False)